<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
from google.colab import drive, output
import IPython
from IPython.display import HTML, Javascript
import json
from IPython.display import clear_output

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:

def switchToScreen(css_filename_, html_filename_, js_filename_):

  clear_output(wait=True)

  if(css_filename_ is None and js_filename_ is None):
      with open(html_filename_, 'r') as file:
        html_content_ = file.read();
        display(IPython.display.HTML(html_content_))
  else:
    with open(css_filename_, 'r') as file:
      css_content_ = file.read();
      display(IPython.display.HTML(css_content_))
    with open(html_filename_, 'r') as file:
      html_content_ = file.read();
      display(IPython.display.HTML(html_content_))
    with open(js_filename_, 'r') as file:
      js_content_ = file.read();
      display(IPython.display.Javascript(js_content_))

In [106]:
def receive_search_string(search_string):

  css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
  html_filename = "/content/drive/My Drive/CloudComputing/project/results.html"
  js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"

  switchToScreen(css_filename, html_filename, js_filename)

  js_code = f"""
    <script>
      document.getElementById("searchbartext").value = "{search_string}";
    </script>
    """
  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_search_string', receive_search_string)

def back_to_home_page():
  css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
  html_filename = "/content/drive/My Drive/CloudComputing/project/index.html"
  js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_home_page', back_to_home_page)

css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
html_filename = "/content/drive/My Drive/CloudComputing/project/index.html"
js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"

switchToScreen(css_filename, html_filename, js_filename)


<IPython.core.display.Javascript object>

In [107]:
def open_statistics_screen():
  html_filename = "/content/drive/My Drive/CloudComputing/project/statistics.html"
  switchToScreen(None, html_filename, None)


# Register the function to be called from JavaScript
output.register_callback('notebook.open_statistics_screen', open_statistics_screen)


html_filename = "/content/drive/My Drive/CloudComputing/project/admin.html"
switchToScreen(None, html_filename, None)